# CNN on the dataset.

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.layers import Dropout

from functools import partial
import re
import os
import time

In [2]:
from file_processing import load_files_list
from file_processing import display_melspectrogram

## Loading the Data

In [3]:
files_list_csv = load_files_list('../Datasets/s_a_d__datasets/melspectrograms_esc50')

In [4]:
df = pd.read_csv('../Datasets/s_a_d__datasets/esc-50_tags.csv')

In [5]:
df. head()

,Unnamed: 0,file,fold,clip_id,take,target
0,0,5-243449-A-14.csv,5,243449,A,14
1,1,2-32515-D-4.csv,2,32515,D,4
2,2,5-210612-A-37.csv,5,210612,A,37
3,3,4-216349-A-34.csv,4,216349,A,34
4,4,1-39923-A-1.csv,1,39923,A,1


In [13]:
# We build a labels list and a targets list.
Images = []
Images_1D = []
Labels = []
i = 0
for file in files_list_csv:
    image_to_be = pd.read_csv('../Datasets/s_a_d__datasets/melspectrograms_esc50/' + file)
    Images_1D.append(image_to_be.to_numpy())
    Images.append(np.stack((Images_1D[i],)*3, axis=-1))
    Labels.append(re.split(r'-|\.', file)[3])
    i=+1

In [5]:
Images = []
Labels = []

for file in files_list_csv:
    image_to_be = pd.read_csv('../Datasets/s_a_d__datasets/experimental/'+file)
    Images.append(image_to_be.to_numpy())
    Labels.append(re.split(r'-|\.', file)[3])

In [6]:
# We turn the lists into numpy arrays.
Images = np.asarray(Images)
Labels = np.asarray(Labels)
# Images_1D = np.asarray(Images_1D)

In [20]:
display_melspectrogram(Images_1D[5])

NameError: name 'sr' is not defined

In [8]:
Labels = pd.get_dummies(Labels)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(Images, Labels, test_size=0.3, stratify=Labels)

In [11]:
y_train

,0,1,10,11,12,13,14,15,16,17,...,45,46,47,48,49,5,6,7,8,9
1967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
954,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
749,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
636,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
508,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X_train.shape

(1400, 128, 217, 3)

## Raw building of a VGG-16 Architecture.

![VGG Structure](https://miro.medium.com/max/700/0*V1muWIDnPVwZUuEv.png)

In [21]:
# The shape we want to as an input is 128, 217
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, activation='relu', padding="SAME")


# Instead of building it by hand maybe we can use:
# from tensorflow.keras.applications.vgg16 import VGG16
model = keras.models.Sequential([
    DefaultConv2D(filters=16, input_shape=[128, 217, 1]),
    DefaultConv2D(filters=16),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=32),
    DefaultConv2D(filters=32),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
#    DefaultConv2D(filters=512),
#    DefaultConv2D(filters=512),
#    DefaultConv2D(filters=512),
#    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.GlobalAveragePooling2D(),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(units=1024, activation='relu'),
    # keras.layers.Dense(units=512, activation='relu'),
    keras.layers.Dense(units=50, activation='softmax')
    
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 128, 217, 32)      320       
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 128, 217, 32)      9248      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 64, 108, 32)       0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 64, 108, 64)       18496     
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 64, 108, 64)       36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 32, 54, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 32, 54, 128)       7

In [22]:
model.compile(optimizer=Adam(lr=0.001, epsilon=1e-8), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
X_train = X_train.reshape(1400, 128, 217, 1)

In [16]:
y_train = pd.get_dummies(y_train)

In [17]:
y_train

,0,1,10,11,12,13,14,15,16,17,...,45,46,47,48,49,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1397,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
batch_size = 30
nb_epochs = 100

In [19]:
callbacks = [EarlyStopping(monitor='accuracy', patience=5, verbose=1, mode='auto')]

In [23]:
model.fit(X_train, y_train, epochs=nb_epochs, batch_size=batch_size, callbacks=callbacks)

Epoch 1/100
47/47 [==============================] - 55s 1s/step - loss: 3.8597 - accuracy: 0.0300
Epoch 2/100
47/47 [==============================] - 59s 1s/step - loss: 3.7710 - accuracy: 0.0379
Epoch 3/100
47/47 [==============================] - 56s 1s/step - loss: 3.7096 - accuracy: 0.0450
Epoch 4/100
47/47 [==============================] - 56s 1s/step - loss: 3.5691 - accuracy: 0.0521
Epoch 5/100
47/47 [==============================] - 55s 1s/step - loss: 3.4287 - accuracy: 0.0800
Epoch 6/100
47/47 [==============================] - 54s 1s/step - loss: 3.2417 - accuracy: 0.1064
Epoch 7/100
47/47 [==============================] - 53s 1s/step - loss: 2.9904 - accuracy: 0.1229
Epoch 8/100
47/47 [==============================] - 53s 1s/step - loss: 2.8322 - accuracy: 0.1743
Epoch 9/100
47/47 [==============================] - 53s 1s/step - loss: 2.6694 - accuracy: 0.2100
Epoch 10/100
47/47 [==============================] - 55s 1s/step - loss: 2.5004 - accuracy: 0.2386
Epoch 11/

## Training on an already built VGG16.

In [10]:
from keras.applications import VGG16

In [11]:
model = VGG16(
    include_top=False, weights='imagenet', input_shape=[128, 217, 3])

In [12]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 217, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 217, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 217, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 108, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 108, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 108, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 54, 128)       0     

In [13]:
model.trainable = False

In [14]:
vgg = Sequential()
for layer in model.layers:
    vgg.add(layer)

In [15]:
vgg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 128, 217, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 217, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 108, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 108, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 108, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 54, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 32, 54, 256)       2

In [16]:
from keras.layers import GlobalAveragePooling2D

In [17]:
vgg.add(GlobalAveragePooling2D())
vgg.add(Dense(units=256, activation='relu')) # Each output has the 'Units' number of neurons
vgg.add(Dropout(0.5))
vgg.add(Dense(units=128, activation='relu'))
vgg.add(Dropout(0.5))
vgg.add(Dense(units=50, activation='softmax'))

In [18]:
vgg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 128, 217, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 217, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 108, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 108, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 108, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 54, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 32, 54, 256)       2

In [1]:
vgg.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

NameError: name 'vgg' is not defined

In [ ]:
vgg.fit(X_train, y_train, epochs=100, batch_size=36)

## Raw building of a ResNet Architecture.

![ResNet Architecture](https://miro.medium.com/max/700/1*S-8Te9IM9tKNznzGJ1JnmQ.png)

In [2]:
from tensorflow.keras.applications import ResNet50

# We may need to resize our images before. Using: images_resized = tf.image.resize(images, [224, 224])
# preprocess_input() can be useful for preprocessing the images  

## Visualizing the CNN

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, GlobalAveragePooling2D

from ann_visualizer.visualize import ann_viz

In [19]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu", input_shape = (128, 216, 1)))
model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
model.add(Dropout(0.1))
model.add(Dense(1024, activation = "relu"))
model.add(Dense(54, activation = "softmax"))


In [20]:
ann_viz(model, title='Simpe CNN')